In [41]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

ENV_HOSTNAME:The-Beast


In [42]:
#Global variables
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_video(video_path, desired_fps=10, target_resolution=(640, 360)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, target_resolution)
        frames.append(frame)

    cap.release()

    frame_interval = int(10 / desired_fps)
    sampled_frames = frames[::frame_interval]

    video_data = np.array(sampled_frames)
    video_data = video_data / 255.0

    # video_data = np.expand_dims(video_data, axis=0)
    # video_data = np.expand_dims(video_data, axis=-1)

    if video_data.shape[0] < 40:
        # Pad the video data with zeros to make it at least 40 frames long
        padding = np.zeros((40 - video_data.shape[0], video_data.shape[1], video_data.shape[2]))
        video_data = np.concatenate((video_data, padding), axis=0)
    elif video_data.shape[0] > 40:
        # Trim the video data to be exactly 40 frames long
        video_data = video_data[:40]

    return video_data

# Example usage:
# video_path = ENV_FOLDER_DATA_PROC + 'clips\\501\\501_524_528.mp4'
# preprocessed_data = preprocess_video(video_path)
# print("Preprocessed video data shape:", preprocessed_data.shape)

# video_dir = ENV_FOLDER_DATA_PROC + 'clips\\595\\'

# for video in os.listdir(video_dir):
#     video_path = video_dir + video
#     preprocessed_data = preprocess_video(video_path)
#     display_frame = np.vstack(preprocessed_data)

#     plt.imshow(display_frame, cmap='gray')
#     plt.axis('off')
#     plt.show()
#     # # display frames
#     # for frame in preprocessed_data:
#     #     cv2.imshow('frame', frame)
#     # break
#     #print("Preprocessed video data shape:", preprocessed_data.shape)

In [131]:
# Create the Data loader

import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import pandas as pd

class VideoDataset(Dataset):
    def __init__(self, video_paths, desired_fps=10, target_resolution=(640, 360), label_type='p'):
        self.video_paths = video_paths
        self.desired_fps = desired_fps
        self.target_resolution = target_resolution
        self.label_type = label_type

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video_data = preprocess_video(video_path, self.desired_fps, self.target_resolution)


        features = torch.tensor(video_data, dtype=torch.float32)
        episode_number = video_path.split('\\')[-2]
        seg_number = video_path.split('_seg')[-1].split('.')[0]
        pickle_path = ENV_FOLDER_DATA_PROC + 'targetdf\\' + episode_number + '.pickle'
        target_df = pd.read_pickle(pickle_path)

        #find row where seg number is equal to seg number and type is equal to label type
        target_row = target_df[(target_df['seg'] == int(seg_number)) & (target_df['type'] == self.label_type)]

        #extract the data from columns 3:43

        audio_data = target_row.iloc[:, 3:44].values

        labels = torch.tensor(audio_data.astype(float).flatten())

        if len(features) != 40 or len(labels) != 41:
            #print(episode_number, seg_number, len(features), len(labels))
            #write to ENV_FOLDER_DATA_PROC + 'invalid_vid_paths.txt'
            with open(ENV_FOLDER_DATA_PROC + 'invalid_vid_paths.txt', 'a') as f:
                f.write(video_path + '\n')
            
            labels = torch.zeros(41)

        return features, labels


from torch.utils.data import DataLoader

clips_dir = ENV_FOLDER_DATA_PROC + 'clips\\'
# Example usage
video_paths = []
for root, dirs, files in os.walk(clips_dir):
    for file in files:
        if file.endswith(".mp4"):
            video_paths.append(os.path.join(root, file))


dataset = VideoDataset(video_paths)
# discard all data points that are None,None
#dataset = [data for data in dataset if data != (None, None)]
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


print(len(dataset))
# Iterate over batches
datapts = 0
for batch in dataloader:
    #print(batch[0].shape, batch[1].shape)
    datapts += batch[0].shape[0]
    print('processed ' + str(datapts) + ' data points')

    # fig, axs = plt.subplots(4, 8, figsize=(16, 10))
    # for i in range(4):
    #     for j in range(8):
    #         idx = i * 8 + j
    #         if idx < len(batch):
    #             frame = batch[idx][20]  # Extracting the first frame
    #             axs[i, j].imshow(frame, cmap='gray')  # Assuming the frame is grayscale
    #             axs[i, j].axis('off')
    #         else:
    #             axs[i, j].axis('off')
    # plt.show()
    # break  # Only display frames from the first batch




34660
processed 32 data points
processed 64 data points
processed 96 data points
processed 128 data points
processed 160 data points
processed 192 data points
processed 224 data points
processed 256 data points
processed 288 data points
processed 320 data points
processed 352 data points
processed 384 data points
processed 416 data points
processed 448 data points
processed 480 data points
processed 512 data points
processed 544 data points
processed 576 data points
processed 608 data points
processed 640 data points
processed 672 data points
processed 704 data points
processed 736 data points
processed 768 data points
processed 800 data points
processed 832 data points
processed 864 data points
processed 896 data points


In [130]:
import os
import shutil

def move_files_to_folder(file_list_path, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Read the list of file paths from the text file
    with open(file_list_path, 'r') as file:
        file_paths = file.readlines()

    # Remove leading/trailing whitespaces and newline characters
    file_paths = [path.strip() for path in file_paths]

    # Move each file to the destination folder
    for file_path in file_paths:
        if os.path.exists(file_path):  # Check if file exists
            file_name = os.path.basename(file_path)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.move(file_path, destination_path)
            print(f"Moved '{file_path}' to '{destination_path}'")
        else:
            print(f"File '{file_path}' not found.")

# Example usage:
file_list_path = ENV_FOLDER_DATA_PROC + 'invalid_vid_paths.txt'  # Path to the text file containing the list of file paths
destination_folder = ENV_FOLDER_DATA_PROC + 'invalid_clips\\'  # Destination folder to move the files to
move_files_to_folder(file_list_path, destination_folder)

Moved 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\clips\519\519_seg1092.mp4' to 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\invalid_clips\519_seg1092.mp4'
Moved 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\clips\540\540_seg1156.mp4' to 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\invalid_clips\540_seg1156.mp4'
Moved 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\clips\514\514_seg1315.mp4' to 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\invalid_clips\514_seg1315.mp4'
Moved 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\clips\567\567_seg258.mp4' to 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\Data_Proc\invalid_clips\567_seg258.mp4'
Moved 'C:\Users\jakes\Documents\COMS 4732 - Computer Vision\W4732CompVisFinal\

In [44]:
#Code to test hypotheses

# 2 Layers of 3D CONV
# 3 Layers of 2D CONV
# Max pooling after each layer
# Batch normalization after each layer
# Flatten
# 3 Fully connected layers

# Leaky ReLU activation function

# adam optimizer
# Loss Function: Cross Entropy

import torch.nn as nn
import torch.nn.functional as F


class VideoConvNet(nn.Module):
    def __init__(self, in_channels=40, num_classes=41):
        super().__init__()
        # TODO: Initialize network layers
        
        self.layer1 = nn.Sequential(
            nn.Conv3d(in_channels=40, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.LeakyReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.LeakyReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(1024*20*20, 4096),
            nn.LeakyReLU(),
            nn.Dropout(0.5))
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.5))
        self.fc3 = nn.Sequential(
            nn.Linear(2048, num_classes))
        







    def forward(self, x):
        # TODO: Implement the forward pass with using the layers defined above
        #       and the proper activation functions

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

vidNet = VideoConvNet()





In [45]:
# Loss function and optimizer

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vidNet.parameters(), lr=0.001)


# Training the network

num_epochs = 10

for epoch in range(num_epochs):
    pass

In [125]:
#print rows of pickle

#video_path = ENV_FOLDER_DATA_PROC + 'clips\\595\\595_seg1434.mp4'
video_path = ENV_FOLDER_DATA_PROC + 'clips\\542\\542_seg1963.mp4'


label_type = 'p'    
episode_number = video_path.split('\\')[-2]
seg_number = video_path.split('_seg')[-1].split('.')[0]
pickle_path = ENV_FOLDER_DATA_PROC + 'targetdf\\' + episode_number + '.pickle'
target_df = pd.read_pickle(pickle_path)

#find row where seg number is equal to seg number and type is equal to label type
#print(target_df)
target_row = target_df[(target_df['seg'] > 1960) & (target_df['type'] == label_type)]
print(target_row)

#extract the data from columns 3:43

audio_data = target_row.iloc[:, 3:].values

# print(episode_number,seg_number, pickle_path, label_type)
# print(audio_data)
# print(audio_data.shape)
# print(audio_data.flatten())
# print(audio_data.flatten().shape)

     epnum   seg type           m          d0          d1          d2  \
585    542  1973    p  147.718303   88.932262   89.800305  161.234423   
590    542  1986    p  147.718303  146.797390  141.325766  147.037827   
595    542  1993    p  147.718303  187.563931  203.546361  227.007572   
605    542  2037    p  147.718303  111.389623  131.937901  193.737167   
610    542  2041    p  147.718303  162.465834  118.514286  116.867901   
...    ...   ...  ...         ...         ...         ...         ...   
1485   542  3214    p  147.718303  129.397717  139.140612  144.711104   
1490   542  3215    p  147.718303         NaN         NaN         NaN   
1495   542  3229    p  147.718303         NaN  194.223374  205.400148   
1500   542  3231    p  147.718303         NaN         NaN  159.558624   
1505   542  3271    p  147.718303         NaN  299.501109  254.987758   

              d3          d4          d5  ...         d30         d31  \
585          NaN  195.670027   94.478089  ...    0